In [ ]:
import nltk
import pandas as pd
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import tensorflow
import copy
import re
import string
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.porter import PorterStemmer
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics  import f1_score,accuracy_score
from sklearn.feature_extraction.text  import TfidfVectorizer
import string as s

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#vax=pd.read_csv("C:/Users/valen/OneDrive/Desktop/UNIMIB/2 ANNO/SOCIAL MEDIA ANALYTICS/progetto/Tweet_vacc.csv", sep=',')
vax=pd.read_csv("/content/drive/MyDrive/SMA/df_tweet_clean.csv", sep=",")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
len(vax)

267665

In [ ]:
vax.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', '_id', 'date', 'is_RT',
       'location', 'text', 'user_id', 'username', 'entities'],
      dtype='object')

In [ ]:
#seleziono solo i tweet da UK
#vax['location']=vax.location.str.lower()
#vax_uk=vax[vax.location=="uk"]

In [ ]:
#già tolti in precedenza
vax=vax[(vax['text'].notna()) | vax['username'].notna()]

## 1. PREPROCESSING

In [ ]:
vax_pp= vax.text[:]

**TEXT NORMALIZATION**

TESTO LOWCASE

In [ ]:
tex_nor=vax_pp.str.lower()

TOKENIZATION

In [ ]:
def tokenization(text):
    lst=text.split()
    return lst

In [ ]:
tex_nor=tex_nor.apply(tokenization)

STOP WORDS

In [ ]:
stop=stopwords.words('english')
stop.append("reuters")
stop.append("afp")

def remove_stopwords(lst):
    new_lst=[]
    for i in lst:
        if i not in stop:
            new_lst.append(i)
    return new_lst

In [ ]:
tex_nor=tex_nor.apply(remove_stopwords)

In [ ]:
#lemmatization
lemmatizer=nltk.stem.WordNetLemmatizer()
def lemmatzation(lst):
    new_lst=[]
    for i in lst:
        i=lemmatizer.lemmatize(i)
        new_lst.append(i)
    return new_lst

In [ ]:
tex_nor=tex_nor.apply(lemmatzation)

In [ ]:
tweet=tex_nor.apply(lambda x: ''.join(i+' '  for i in x))

In [ ]:
vax["text"]=tweet

In [ ]:
vax

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,date,is_RT,location,text,user_id,username,entities
0,6,6,6.0,5fcbb342ce2a8e9b3d4d23f9,2020-12-05 16:20:13,False,New York,head pfizer research: covid vaccine female ste...,1.844636e+08,🇺🇸 USA Strong! 🇺🇸,NaN
1,7,7,7.0,5fcbb342ce2a8e9b3d4d23fa,2020-12-05 16:20:13,False,NaN,@lads1234 @paigebmacdonald vaccine different i...,2.878667e+09,Glenn Mellen,NaN
2,11,11,11.0,5fcbb343ce2a8e9b3d4d23fe,2020-12-05 16:20:13,False,NaN,great letter @thomasjtobin1 flock covid vaccin...,3.194793e+09,Michael R. Heinlein,NaN
3,19,19,19.0,5fcbb344ce2a8e9b3d4d2406,2020-12-05 16:20:14,False,NaN,"dark winter: biological agents, act war. read ...",1.243255e+18,Kraken - Elect RealEyes,NaN
4,20,20,20.0,5fcbb344ce2a8e9b3d4d2407,2020-12-05 16:20:14,False,United States,@missy04612576 hmm. j larry shakelford seems l...,7.734929e+17,⚡️💎,NaN
...,...,...,...,...,...,...,...,...,...,...,...
267660,842501,41683,41683.0,5fe074a73f4fea8a8e641442,2020-12-21 10:10:42,False,NaN,@rarebreed31 rawing stranger dont speak vaccine.,1.077095e+18,Melv.Erotic,NaN
267661,842520,41702,41702.0,5fe074aa3f4fea8a8e641455,2020-12-21 10:10:45,False,NaN,"longest queue seen year waitrose west byfleet,...",2.290340e+08,Gary,NaN
267662,842529,41711,41711.0,5fe074ab3f4fea8a8e64145e,2020-12-21 10:10:46,False,"Atlanta, GA",problem. big problem.,9.871399e+08,Jim Witherspoon,NaN
267663,842531,41713,41713.0,5fe074ac3f4fea8a8e641460,2020-12-21 10:10:46,False,"Cape Town, South Africa",govt monitor anti-vax activity social medium c...,1.106644e+18,David Gautier,NaN


In [ ]:
len(vax)

267665

In [ ]:
#elimino righe dove username contiene la stringa bot 
vax2=vax[~vax['username'].str.contains(r'bot')]

In [ ]:
len(vax2)

267410

**MENZIONI**

In [ ]:
def find_mentions(x):
    p = re.compile(r'@([^\s:]+)')
    return ', '.join(p.findall(x))

In [ ]:
vax["@mentions"] = vax["text"].apply(find_mentions)

ANALISI DELLE MENZIONI

In [ ]:
menzioni = pd.DataFrame(vax["@mentions"].str.split(",",expand=True))

In [ ]:
menzioni=menzioni.melt()

In [ ]:
menzioni = menzioni[pd.notnull(menzioni['value'])]

In [ ]:
menzioni = pd.DataFrame(menzioni["value"].value_counts()).reset_index()

In [ ]:
menzioni.rename(columns = {'index': "Persone citate", 
                           'value':"Numero di citazioni"}, inplace = True)

In [ ]:
mentions_count = pd.DataFrame(menzioni[menzioni["Numero di citazioni"] > 5]).reset_index()

In [ ]:
mentions_count1=mentions_count[1:]

In [ ]:
mentions_count1

,index,Persone citate,Numero di citazioni
1,1,realdonaldtrump,8657
2,2,us_fda,8414
3,3,stevefda.,7591
4,4,realdonaldtrump,5647
5,5,pfizer,2665
...,...,...,...
8439,8439,ronmilnerboodle,6
8440,8440,ariadnelabs,6
8441,8441,kingscourt0007,6
8442,8442,ugursahinprf,6


In [ ]:
from plotly import express

In [ ]:
import plotly.express as px
#tips = px.mention_count1.tips()
fig = px.bar(mentions_count1[0:25], y="Persone citate", x = "Numero di citazioni" ,title = "Le persone più menzionate", 
             orientation ="h", text = "Numero di citazioni", )
fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    #paper_bgcolor="white"
)
fig.show()

**HASHTAG**

In [ ]:
def find_hashtags(x):
    p = re.compile(r'#([^\s:]+)')
    return ', '.join(p.findall(x))

In [ ]:
vax["#hashtags"] = vax["text"].apply(find_hashtags)

In [ ]:
hashtags = pd.DataFrame(vax["#hashtags"].str.split(",",expand=True))

In [ ]:
hashtags=hashtags.melt()

In [ ]:
hashtags = hashtags[pd.notnull(hashtags['value'])]

In [ ]:
hashtags = pd.DataFrame(hashtags["value"].value_counts()).reset_index()

In [ ]:
hashtags.head(20)

,index,value
0,,738428
1,covid19,21641
2,vaccine,9265
3,covid19,8936
4,vaccine,4965
5,covidvaccine,4327
6,covidvaccine,4021
7,vaccines,2899
8,coronavirus,2462
9,coronavirus,2294


In [ ]:
hashtags.rename(columns = {'index': "Hashtag_usati", 
                           'value':"Numero"}, inplace = True)

In [ ]:
tag_count = pd.DataFrame(hashtags[hashtags["Numero"] > 5]).reset_index()

In [ ]:
tag_count1=tag_count[1:]

In [ ]:
tag_count1

,index,Hashtag usati,Numero
1,1,covid19,21641
2,2,vaccine,9265
3,3,covid19,8936
4,4,vaccine,4965
5,5,covidvaccine,4327
...,...,...,...
4168,4168,ship,6
4169,4169,5liveyourcall,6
4170,4170,privacy,6
4171,4171,globalism,6


In [ ]:
tag_count1['Hashtag usati']=tag_count1['Hashtag usati'].str.strip()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
tag_count1= tag_count1.replace(" ", "")

In [ ]:
tag_count1

,index,Hashtag usati,Numero
1,1,covid19,21641
2,2,vaccine,9265
3,3,covid19,8936
4,4,vaccine,4965
5,5,covidvaccine,4327
...,...,...,...
4168,4168,ship,6
4169,4169,5liveyourcall,6
4170,4170,privacy,6
4171,4171,globalism,6


In [ ]:
tag=tag_count1.groupby(['Hashtag usati']).agg({'Numero':'sum'}).reset_index()

In [ ]:
tag=tag.sort_values(by='Numero', ascending=False)

In [ ]:
import plotly.express as px
tips = px.data.tips()
fig = px.bar(tag[0:25], 
             y="Hashtag usati", 
             x = "Numero", 
             title = "Hashtag più usati", 
             orientation ="h", 
             text = "Hashtag usati")
fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    paper_bgcolor="white"
)
fig.show()

In [ ]:
#tolgo hashtag #covid19
import plotly.express as px
tips = px.data.tips()
fig = px.bar(tag[1:26], 
             y="Hashtag usati", 
             x = "Numero", 
             title = "Hashtag più usati", 
             orientation ="h", 
             text = "Hashtag usati")
fig.update_layout(
    margin=dict(l=50, r=50, t=50, b=50),
    paper_bgcolor="white"
)
fig.show()

**PROVENIENZA DEI TWEET**

In [ ]:
locations=vax.groupby(["location"], as_index=False).count()

In [ ]:
location_sort=locations.sort_values(by=['username'],ascending=False)

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(go.Bar(
    x=location_sort["location"][:10],y=location_sort['username'][:30],
    marker={'color': location_sort['username'][:30], 
    'colorscale': 'blues'},  
    #text=all_tweet_location['count'][:30],
    textposition = "outside",
))
fig.update_layout(title_text='Top Countries with most tweets',xaxis_title="Countries",
                  yaxis_title="Number of Tweets ",template="plotly_dark",height=700,title_x=0.5)

fig.show()

In [ ]:
vax.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,_id,date,is_RT,location,text,user_id,username,entities
0,6,6,6.0,5fcbb342ce2a8e9b3d4d23f9,2020-12-05 16:20:13,False,New York,head pfizer research: covid vaccine female ste...,1.844636e+08,🇺🇸 USA Strong! 🇺🇸,NaN
1,7,7,7.0,5fcbb342ce2a8e9b3d4d23fa,2020-12-05 16:20:13,False,NaN,@lads1234 @paigebmacdonald vaccine different i...,2.878667e+09,Glenn Mellen,NaN
2,11,11,11.0,5fcbb343ce2a8e9b3d4d23fe,2020-12-05 16:20:13,False,NaN,great letter @thomasjtobin1 flock covid vaccin...,3.194793e+09,Michael R. Heinlein,NaN
3,19,19,19.0,5fcbb344ce2a8e9b3d4d2406,2020-12-05 16:20:14,False,NaN,"dark winter: biological agents, act war. read ...",1.243255e+18,Kraken - Elect RealEyes,NaN
4,20,20,20.0,5fcbb344ce2a8e9b3d4d2407,2020-12-05 16:20:14,False,United States,@missy04612576 hmm. j larry shakelford seems l...,7.734929e+17,⚡️💎,NaN


**SCARICO IL DATASET PULITO**

In [ ]:
vax.to_csv('/content/drive/MyDrive/SMA/vax2.csv')